In [1]:
# Pip install method (recommended)

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.196  Python-3.10.9 torch-2.2.2+cpu CPU (11th Gen Intel Core(TM) i3-1115G4 3.00GHz)
Setup complete  (4 CPUs, 7.8 GB RAM, 120.9/146.5 GB disk)


In [2]:
from ultralytics import YOLO

from IPython.display import display, Image

In [3]:
from roboflow import Roboflow
rf = Roboflow(api_key="10uIUjTCg7s1cs23NXvV")
project = rf.workspace("hendro-wunga").project("runspose")
version = project.version(2)
dataset = version.download("yolov8")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to RunsPose-2 in yolov8:: 100%|██████████| 2203/2203 [00:06<00:00, 315.14it/s]


In [4]:
%cd {HOME}
task=detect mode=train model=yolov8s.pt data={dataset.location}/data.yaml epochs=25 imgsz=800 plots=True


SyntaxError: invalid syntax (3581783326.py, line 2)

In [ ]:

def sharpen_image_with_laplacian(image):
    # Apply the Laplacian operator to detect edges\
    color_image = cv.cvtColor(image, cv.COLOR_GRAY2BGR)
    laplacian = cv.Laplacian(image, cv.CV_64F)

    # Calculate the sharpened image by adding the Laplacian image to the original image
    # Normalize the Laplacian result to the range [0, 255]
    laplacian_norm = cv.normalize(laplacian, None, alpha=0, beta=255, norm_type=cv.NORM_MINMAX, dtype=cv.CV_8U)

    # Convert Laplacian result back to 3-channel format
    laplacian_rgb = cv.merge((laplacian_norm, laplacian_norm, laplacian_norm))

    # Calculate the sharpened image by adding the Laplacian-enhanced image to the original image
    sharpened_image = cv.addWeighted(color_image, 1.5, laplacian_rgb, -0.5, 0)

    return sharpened_image


list_files("../Dataimage/Train")